In [1]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.auth import OAuth2ClientCredentials
from dlt.sources.helpers.rest_client.paginators import JSONLinkPaginator
from dlt.sources.rest_api import rest_api_source

In [2]:
auth = OAuth2ClientCredentials(
    access_token_url=dlt.config["sources.pbi.access_token_url"],
    client_id=dlt.secrets["sources.pbi.client_id"],
    client_secret=dlt.secrets["sources.pbi.client_secret"],
    access_token_request_data={
        "scope":dlt.config["sources.pbi.scope"],
        "grant_type":"client_credentials"
    }
)

In [3]:
@dlt.source
def all_data():
    return [get_ws]

In [4]:
@dlt.resource(table_name="workspaces",write_disposition="replace")
def get_ws():
    source_config= {
        "client": {
            "base_url":dlt.config["sources.pbi.base_url"],
            "auth":auth
        },
        "resources": [
            {
                "name":"workspaces",
                "primary_key":"id",
                "endpoint": {
                    "path":"admin/groups",
                    "params":{
                        "$top": 5000,
                        "$expand":"users,reports,datasets"
                    },
                },
                "max_table_nesting":5
            },
        ],
    }

    source = rest_api_source(source_config)
    for resource in source:
        yield resource

In [8]:
pipeline = dlt.pipeline(
        pipeline_name="pbijeudi",
        destination=dlt.destinations.duckdb("Db/pbi.db"),
        dataset_name="raw"
    )

In [9]:
load_info = pipeline.run(all_data())
print(load_info)

2025-05-12 23:34:45,251|[WARNING]|25868|26572|dlt|client.py|detect_paginator:312|Fallback paginator used: SinglePagePaginator at 1b745149bd0. Please provide right paginator manually.


Pipeline pbijeudi load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset raw
The duckdb destination used duckdb:///c:\Dev\Lobellia\Db/pbi.db location to store data
Load package 1747085683.450759 is LOADED and contains no failed jobs
